# LangChain for LLM Application Development

LLM 애플리케이션용 오픈 소스 개발 프레임워크

Python 및 JavaScript(TypeScript) 패키지

구성 및 모듈성에 중점을 둡니다.

핵심 가치에는 다음이 있습니다.
1. 모듈식 구성 요소
2. 사용 사례: 구성 요소를 결합하는 일반적인 방법

Key value adds:
1. Modular components
2. Use cases: Common ways to combine components

* Models
 - LLMs: 20+ integrations
 - Chat Models
 - Text Embedding Models: 10+ integrations
 
* Prompts
 - Prompt templates
 - Output Parsers: 5+ implementations
   - Retry/fixing logic
 - Example Selectors: 5+ implementations
 
* Indexes
 - Document loaders: 50+ implementations
 - Text Splitters: 10+ implementations
 - Vector stores: 10+ integrations
 - Retriviers: 5+ integrations/implementations
 
* Chains
 - Prompt + LLM + Output parsing
 - Can be used as building blocks for longer chains
 - More application specific chains: 20+ types
 
* Agents
 - Agent Types: 5+ types
   - Algorithms for getting LLMs to use tools
 - Agent Toolkits: 10+ implementations
   - Agents armed with specific tools for a specific application


## Models, Prompts and Parsers

여기에서는 모델, 프롬프트 및 구문 분석기를 다룰 것입니다. 모델은 많은 것을 뒷받침하는 언어 모델을 나타냅니다. 프롬프트는 모델에 전달할 입력을 만드는 스타일을 나타냅니다. 그리고 파서는 반대쪽에 위치하며, 모델의 출력을 가져오고 더 구조화된 형식으로 구문 분석하여 다운스트림 작업을 수행할 수 있습니다.

LLM을 사용하여 애플리케이션을 구축할 때 재사용 가능한 모델이 있는 경우가 많습니다. 우리는 반복적으로 모델을 프롬프트하고 출력을 구문 분석하므로 LangChain은 이러한 유형의 작업을 수행하기 위한 손쉬운 추상화 세트를 제공합니다.

## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [20]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 이것는 모델 출력의 무작위성 정도를 말합니다.[0~1]
    )
    return response.choices[0].message.content

In [22]:
get_completion("What is 1+1?")

'1+1 equals 2.'

이제 모델 프롬프트 및 파서에 대한 LangChain 추상화에 동기를 부여하기 위해 영어가 아닌 언어로 고객으로부터 이메일을 받았다고 가정해 보겠습니다.
이것이 접근 가능한지 확인하기 위해 제가 사용할 다른 언어는 영국 해적 언어입니다.

In [25]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [27]:
style = """American English \
in a calm and respectful tone
"""

In [29]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [31]:
response = get_completion(prompt)

In [32]:
response

"Oh man, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to top it off, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, buddy."

## Chat API : LangChain

Let's try how we can do the same using LangChain.

### Model

In [40]:
from langchain_openai import ChatOpenAI

In [42]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000273A407DE20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000273A407F6E0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

### Prompt template

In [45]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [47]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [49]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [51]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [53]:
customer_style = """American English \
in a calm and respectful tone
"""

In [55]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [57]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [59]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [61]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [63]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

C:\Users\User\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [65]:
print(customer_response.content)

I am really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend.


In [67]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [69]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [71]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [73]:
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the cost o' cleanin' yer galley due to yer own negligence in misusin' yer blender. Ye forgot to secure the lid afore startin' the blender, so it be on ye, matey! Fair winds to ye! Farewell!


<img src="why_prompt_template.PNG" width="400">

<img src="prompt_output.PNG" width="400">


## Output Parsers

LLM 출력이 다음과 같이 표시되는 방법을 정의하는 것으로 시작하겠습니다.

In [77]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [79]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [81]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'))]


In [123]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}


In [84]:
type(response.content)

str

In [87]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### (New) A Simple output parsing (by Jun Ji)

In [125]:
json_text = json.loads(f'''{response.content}''')

In [127]:
json_text.get('delivery_days')

2

### (Old) Parse the LLM output string into a Python dictionary

In [90]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [92]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [94]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [96]:
format_instructions = output_parser.get_format_instructions()

In [98]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [100]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [102]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [104]:
response = chat(messages)

In [105]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [106]:
output_dict = output_parser.parse(response.content)

In [110]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [112]:
type(output_dict)

dict

In [114]:
output_dict.get('delivery_days')

2